In [ ]:
import numpy as np
from hmmlearn import hmm
import alfabeto_data.dissertation_images as di
import alfabeto_data.hmm_threading as ht
from alfabeto_data.harmonic_function_data import ordered_numeral_list
from alfabeto_sources.all_sources import *
from alfabeto_sources import *
from alfabeto_code.AlfabetoConverter import transposed_pc_chords_noMMD
from Continuo.ContinuoConverter import figure_intervals_pc
import csv, copy, time

from sklearn.externals import joblib
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
from scipy.spatial.distance import euclidean, cdist

In [ ]:
def many_hmm(corpus_dict, mode, n_functions, crd_type):
    total_estemit = []
    corpus_names = []
    all_numerals = ['I', 'i', 'bII', 'bii', 'II', 'ii', 'bIII', 'biii', 'III', 'iii', 'IV', 'iv',
                    '#IV', '#iv', 'V', 'v', 'bVI', 'bvi', 'VI', 'vi', 'bVII', 'bvii', 'VII', 'vii']
    all_numerals = ordered_numeral_list
    for j, x in corpus_dict.items():
        temp_emit = []
        corpus_names.append(j)
        X = '_'
        if type(x) is tuple:
            X = di.python_hmm_corpus(x, mode)
        elif type(x) is list:
            X = di.python_hmm(x, mode, crd_type)
        K = n_functions
        Y = hmm.MultinomialHMM(n_components=K, verbose=False,n_iter=500,tol=.001)
        Y.n_features=len(X[2])
        a = Y.fit(X[0], X[1])
        ESTTR11 = a.transmat_
        TOKEN_NAMES=X[2]
        ESTEMIT11 = a.emissionprob_
        
        #trying to insert unused numerals...
        ordered_emit = [] #T-S-D
        numerals_in = [] #indices of numerals in the corpus
        numerals_not = [] #indices of numerals not in the corpus
        for numeral in all_numerals:
            if numeral in TOKEN_NAMES:
                numerals_in.append(all_numerals.index(numeral))
            else:
                numerals_not.append(all_numerals.index(numeral))
        for n_nu in numerals_not:
            ESTEMIT11 = np.insert(ESTEMIT11, n_nu, np.array([0 for numb in range(K)]), 1)
        
        
        for r in range(len(all_numerals)):
            if all_numerals[r] == 'I':
                row = ESTEMIT11[:,r]
                most = max(row)
                for f in range(len(row)):
                    if row[f] == most:
                        temp_emit.append(ESTEMIT11[f])
            if all_numerals[r] == 'IV':
                row = ESTEMIT11[:,r]
                most = max(row)
                for f in range(len(row)):
                    if row[f] == most:
                        temp_emit.append(ESTEMIT11[f])
            if all_numerals[r] == 'V':
                row = ESTEMIT11[:,r]
                most = max(row)
                for f in range(len(row)):
                    if row[f] == most:
                        temp_emit.append(ESTEMIT11[f])
        temp_emit_2 = []
        for xx in temp_emit:
            for yy in xx:
                temp_emit_2.append(yy)
        
        total_estemit.append(temp_emit_2)


#         print(len(total_estemit[0]))
        print('done one')
    return total_estemit, corpus_names

In [ ]:
all_data = []
for corpus in ['alfabeto_continuo', 'bach', 'palestrina']:
    for mode in ['major', 'minor']:
        for mc_type in range(5):
            flipped_data = {}
#             corpus = 'alfabeto_continuo'
            K_range = 5
#             mode = 'major'
            KK = [2, 3, 4, 5, 6, 8, 9, 12]
            for K in range(2, K_range+1):
            # for K in KK:
                all_data = []
                all_labels = []
                l_number = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/0.pkl' % (corpus, mode, K, mc_type))['test'][0]
                print(len(l_number))
                l_numeral = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/0.pkl' % (corpus, mode, K, mc_type))['train'][2]
                for l in l_number:
                    if l[0] < len(l_numeral):
                        all_labels.append(l_numeral[l[0]])
                    else:
                        all_labels.append('o')
            #             print('one didn\'t fit...')
                for x in range(500):
                    temp_data = []  
                    c = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/%s.pkl' % (corpus, mode, K, mc_type, x))['decode']
            #         print('decoded')
                    for xx in c[1]:
                        temp_data.append(xx)        
                    all_data.append(temp_data)
#                 for x in np.array(all_data):
#                     print(len(x))
                numpied = np.swapaxes(np.array(all_data), 0, 1)
            #     print(numpied.shape)
                DIST = []
                st = time.time()
                DIST = cdist(numpied, numpied, 'hamming')
            #     DIST = (cdist(numpied, numpied, 'hamming')*len(l_number))**2 #Quinn/White
            #     print('DIST', K, time.time()-st)
            #     flipped_data.append(DIST)
                st2 = time.time()
                flipped_data[K] = di.k_means_simple(DIST, K, all_labels)
            #     print_labels = ['$'+x+'$' for x in all_labels]
            #     di.k_means_data(DIST, K, print_labels, '/home/daniel/Desktop/hmmkmeans/%s_%s.pdf' % (corpus, K))
            #     print('kmeans took', time.time()-st2)
                print(corpus, mode, K, mc_type, '-->', flipped_data[K])
            all_data.append({(corpus, mode, mc_type): flipped_data})
joblib.dump(all_data, '/home/daniel/Desktop/silhouette_data.pkl', compress=6)

In [ ]:
for k in range(2, 5):
    for mc in range(5):
        xlen = []
        for x in range(50):
            xx = joblib.load('hmm2/hmm_mc/alfabeto_continuo/major/%s/%s/%s.pkl' % (k, mc, x))
            xlen.append(len(xx['decode'][1]))
        if len(set(xlen))>1:
            print('yikes!', k, mc)
            print(xlen)

In [ ]:
import ipyparallel as ipp
from sklearn.externals import joblib
rc = ipp.Client()
dview = rc[:]

dview.execute('import numpy as np')
dview.execute('import time')
dview.execute('from sklearn.externals import joblib')
dview.execute('from scipy.spatial.distance import cdist')
dview.execute('import alfabeto_data.dissertation_images as di')


@dview.parallel(block=True)
def silhouette_hmm(mc):
    corpus_name = 'alfabeto_continuo'
    mode_name = 'major'
    K_range = 15
    flipped_data = {}
    for K in range(2, K_range+1):
        all_data = []
        all_labels = []
        l_number = joblib.load('hmm2/hmm_mc/%s/%s/%s/%s/0.pkl' % (corpus_name, mode_name, K, mc))['test'][0]
        l_numeral = joblib.load('hmm2/hmm_mc/%s/%s/%s/%s/0.pkl' % (corpus_name, mode_name, K, mc))['train'][2]
        for l in l_number:
            if l[0] < len(l_numeral):
                all_labels.append(l_numeral[l[0]])
            else:
                all_labels.append('o')
        #             print('one didn\'t fit...')
        for x in range(50):
            temp_data = []
            print(x)
            c = joblib.load('hmm2/hmm_mc/%s/%s/%s/%s/%s.pkl' % (corpus_name, mode_name, K, mc, x))['decode']
    #         print('decoded')
            for xx in c[1]:
                temp_data.append(xx)
            all_data.append(temp_data)
        numpied = np.swapaxes(np.array(all_data), 0, 1)
    #     print(numpied.shape)
        st = time.time()
        DIST = cdist(numpied, numpied, 'hamming')
    #     DIST = (cdist(numpied, numpied, 'hamming')*len(l_number))**2 #Quinn/White
        st2 = time.time()
        flipped_data[K] = di.k_means_simple(DIST, K, all_labels)
    #     print_labels = ['$'+x+'$' for x in all_labels]
    #     di.k_means_data(DIST, K, print_labels, '/home/daniel/Desktop/hmmkmeans/%s_%s.pdf' % (corpus, K))
    #     print('kmeans took', time.time()-st2)
        print(K, '-->', flipped_data[K])
    return flipped_data

j = silhouette_hmm.map([mc for mc in range(5)])
joblib.dump(j, 'silhouette_alfabeto_data.pkl')




In [ ]:
badones = []
for corpus in ['alfabeto_continuo', 'bach', 'palestrina']:
    for mode in ['major', 'minor']:
        for K in range(2, 15):
            for mc_type in range(5):
                for krange in range(500):
                    try:
#                         print('fine')
                        x = joblib.load('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/%s.pkl' % (corpus, mode, K, mc_type, krange))
                    except EOFError:
                        badones.append((corpus, mode, K, mc_type, krange))
                        print((corpus, mode, K, mc_type, krange))



In [ ]:
joblib.dump(badones, '/home/daniel/Desktop/badones.pkl')
import os
for x in badones:
    os.remove('/home/daniel/Desktop/hmm_mc/%s/%s/%s/%s/%s.pkl' % (x[0], x[1], x[2], x[3], x[4]))